In [5]:
import pandas as pd
import json
from glob import glob
import os
import numpy as np
from collections import defaultdict
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

In [6]:
def from_params(path):
    filenames = glob(path + "/*")

    records = []
    for f in filenames:
        with open(f) as json_file:
            tmp = json.load(json_file)
            tmp["id"] = f[f.rfind("/")+1:-4]
            records.append(tmp)
    
    return pd.DataFrame(records)


def tabulate_events(dpath):
    summary_iterators = [EventAccumulator(os.path.join(dpath, dname)).Reload() for dname in os.listdir(dpath)]

    tags = summary_iterators[0].Tags()['scalars']

    for it in summary_iterators:
        assert it.Tags()['scalars'] == tags

    out = defaultdict(list)

    for si in summary_iterators:
        for tag in tags:
            out[tag].append(si.Scalars(tag)[-1].value)
        out["step"].append(si.Scalars(tag)[-1].step)

    return out


def from_tb(dpath):
    dirs = os.listdir(dpath)

    d = tabulate_events(dpath)
    return pd.DataFrame(d)

In [11]:
params = from_params("experiments/parameters")
metrics = from_tb('experiments/logs/')
pd.concat([params, metrics], axis=1).sort_values("train/f1")

,batch_size,bn,command,data_tag,dropout,epochs,hidden_size,learning_rate,mega_epochs,num_layers,id,train/loss,train/f1,train/precision,train/recall,val/loss,val/f1,val/precision,val/recall,step
4,64,True,train_rnn.py --hidden_size 512 --learning_rate...,rc1-rc2-5k,0.0,1000,512,0.010,NaN,3,de00,1.386295,0.495180,0.500321,0.490144,1.386294,0.000000,0.000000,0.0000,15
0,64,True,train_rnn.py --hidden_size 32 --learning_rate ...,rc1-rc2-5k,0.0,200,32,0.010,30.0,1,3b68,1.075540,0.716208,0.732769,0.700379,1.390256,0.664834,0.506494,0.9672,6000
1,64,True,train_rnn.py --hidden_size 32 --learning_rate ...,rc1-rc2-5k,0.0,200,32,0.010,30.0,2,2c0d,0.889976,0.795055,0.791723,0.798414,1.513147,0.004374,0.366667,0.0022,6000
3,64,True,train_rnn.py --hidden_size 32 --learning_rate ...,rc1-rc2-5k,0.0,200,32,0.010,30.0,1,d4aa,0.918781,0.815164,0.801597,0.829199,1.735308,0.013774,0.426829,0.0070,6000
2,64,True,train_rnn.py --hidden_size 32 --learning_rate ...,rc1-rc2-5k,0.0,200,32,0.010,30.0,1,a01a,0.817985,0.820228,0.809852,0.830873,1.470305,0.335288,0.445476,0.2688,6000
5,64,True,train_rnn.py --hidden_size 128 --learning_rate...,rc1-rc2-5k,0.0,1000,128,0.001,NaN,3,d586,0.184384,0.965935,0.962144,0.969756,1.969085,0.663391,0.503523,0.9720,16


In [17]:
tmp = pd.concat([params, metrics], axis=1).sort_values("train/f1", ascending=False).reset_index(drop=True)
print(tmp["train/f1"][0])
tmp["command"][0]

0.9659353494644165


'train_rnn.py --hidden_size 128 --learning_rate 0.001 --num_layers 3 --data_tag rc1-rc2-5k --bn 1'